In [1]:
import tensorflow as tf
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from tabulate import tabulate
from mpl_toolkits.mplot3d import Axes3D

def make_candidate_functions(data, max_derivation=3, max_degree=3, functions={}):
    """Returns the array and the number of transformations"""
    def make_example(row, state_space):
        example = [np.ones(state_space), row]
        for k in range(max_derivation):
            example.append(np.gradient(example[k + 1]))
        return example
    assert len(data.shape) == 2
    state_space = data.shape[1]
    result = []
    for example in data:
        row = []
        # making an array with [1, x, y, ...]
        example = make_example(example, state_space)
        for indexes in combinations_with_replacement(range(max_derivation + 2), max_degree):
            product = np.take(example, indexes, axis=0).prod(axis=0)
            row.append(product)
        result.append(row)
    return np.transpose(np.array(result, dtype=np.float32), (0, 2, 1)), len(row)

def flatten_data(augmented, targets):
    augmented = augmented.reshape(augmented.shape[0]*augmented.shape[1], augmented.shape[2])
    targets = targets.reshape(targets.shape[0]*targets.shape[1], 1)
    return augmented, targets

In [2]:
import codebase
from codebase.data_generation import *
from codebase.sparse_regression import *
from codebase.utils import *

In [6]:
heat = sio.loadmat('C:/Users/kipr/Downloads/heat.mat')['simple']
hidden = sio.loadmat('C:/Users/kipr/Downloads/produced_data.mat')['solution']

In [13]:
targets, m = make_targets(hidden, derivative=True)
augmented, _ = make_candidate_functions(m, max_derivation=(deriv_deg:=2), max_degree=2)
augmented, targets = flatten_data(augmented, targets)

In [15]:
weights, _ = sparse_regression(augmented, targets, cutoff=1e-1)

print(f'mse={((targets - tf.matmul(augmented, weights))**2).numpy().mean()}')
show_weights(weights, derivative=True, pde=deriv_deg)

mse=0.0002315423625987023


| function      |   $\dot{u}$ |
|:--------------|------------:|
| $1$           |      0      |
| $ u$          |      0      |
| $ u_x$        |      0      |
| $ u_{xx}$     |     10.8436 |
| $ u^2$        |      0      |
| $ u u_x$      |      0      |
| $ u u_{xx}$   |      0      |
| $ u_x^2$      |      0      |
| $ u_x u_{xx}$ |      0      |
| $ u_{xx}^2$   |      0      |